# Fire DataCube Preperation

In [1]:
import os
import re
import json
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Dict, Optional, Union, Tuple

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import box
import rasterio
from rasterio.windows import Window
from rasterio.windows import bounds as window_bounds
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.mask import mask
from tqdm import tqdm

def create_reference_grid(
    ca_boundary: str,
    out_path: str,
    dst_crs: str = "EPSG:3310",
    dst_res: float = 100.0,
    compress: str = "lzw"
) -> str:
    """
    Create a reference grid that covers California at specified resolution.
    This grid will be used to align ALL rasters (static and dynamic).
    
    Returns: path to the reference grid raster
    """
    print(f"[Creating Reference Grid]")
    ca = gpd.read_file(ca_boundary)
    if ca.crs is None:
        raise ValueError("CA boundary has no CRS")
    
    ca = ca.to_crs(dst_crs)
    bounds = ca.total_bounds  # minx, miny, maxx, maxy
    
    # Calculate dimensions
    width = int(np.ceil((bounds[2] - bounds[0]) / dst_res))
    height = int(np.ceil((bounds[3] - bounds[1]) / dst_res))
    
    # Create transform
    from rasterio.transform import from_origin
    transform = from_origin(bounds[0], bounds[3], dst_res, dst_res)
    
    # Create empty reference raster
    profile = {
        'driver': 'GTiff',
        'height': height,
        'width': width,
        'count': 1,
        'dtype': 'float32',
        'crs': dst_crs,
        'transform': transform,
        'nodata': -9999.0,
        'compress': compress
    }
    
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with rasterio.open(out_path, 'w', **profile) as dst:
        dst.write(np.zeros((height, width), dtype='float32'), 1)
    
    print(f"  ✓ Reference grid created: {width}x{height} px at {dst_res}m resolution")
    print(f"  ✓ Saved to: {out_path}")
    return out_path


def align_raster_to_reference(
    src_raster: str,
    reference_grid: str,
    out_path: str,
    ca_boundary: str,
    compress: str = "lzw"
) -> str:
    """
    Reproject and snap source raster to reference grid, then clip to CA boundary.
    """
    print(f"  Processing: {Path(src_raster).name}")
    
    # Convert to Path object and create parent directories
    out_path = Path(out_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    
    # Step 1: Reproject to match reference grid
    with rasterio.open(reference_grid) as ref, rasterio.open(src_raster) as src:
        profile = ref.profile.copy()
        profile.update({'compress': compress})
        
        temp_path = str(out_path.parent / (out_path.name + ".temp.tif"))
        with rasterio.open(temp_path, 'w', **profile) as dst:
            reproject(
                source=rasterio.band(src, 1),
                destination=rasterio.band(dst, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=ref.transform,
                dst_crs=ref.crs,
                resampling=Resampling.nearest,  # Use nearest for categorical data
                src_nodata=src.nodata,
                dst_nodata=ref.nodata
            )
    
    # Step 2: Clip to CA boundary
    ca = gpd.read_file(ca_boundary)
    with rasterio.open(reference_grid) as ref:
        if ca.crs != ref.crs:
            ca = ca.to_crs(ref.crs)
    
    geoms = [geom.__geo_interface__ for geom in ca.geometry if geom is not None]
    
    with rasterio.open(temp_path) as src:
        out_image, out_transform = mask(src, geoms, crop=True, nodata=src.nodata, filled=True)
        out_meta = src.meta.copy()
        out_meta.update({
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform,
            "compress": compress
        })
        
        with rasterio.open(str(out_path), "w", **out_meta) as dst:
            dst.write(out_image)
    
    # Clean up temp file
    if os.path.exists(temp_path):
        os.remove(temp_path)
    
    print(f"    ✓ Aligned and clipped: {out_path.name}")
    return str(out_path)


def discover_dynamic_rasters(
    dynamic_folders: Dict[str, str],  # {variable_name: folder_path}
    pattern: str = "*.tif"
) -> Dict[str, List[str]]:
    """
    Discover all .tif files in each dynamic variable folder.
    
    Args:
        dynamic_folders: {variable_name: folder_path containing all tifs for that variable}
        pattern: file pattern to match (default: "*.tif")
    
    Returns:
        {variable_name: [sorted list of paths]}
    """
    dynamic_rasters = {}
    for var_name, folder_path in dynamic_folders.items():
        folder = Path(folder_path)
        if not folder.exists():
            raise ValueError(f"Folder not found: {folder_path}")
        
        tif_files = sorted(folder.glob(pattern))
        if not tif_files:
            raise ValueError(f"No .tif files found in {folder_path}")
        
        dynamic_rasters[var_name] = [str(p) for p in tif_files]
        print(f"  {var_name}: found {len(tif_files)} files")
    
    return dynamic_rasters


def align_all_rasters(
    static_rasters: Dict[str, str],  # {variable_name: path}
    dynamic_rasters: Dict[str, List[str]],  # {variable_name: [path1, path2, ...]}
    landcover_raster: Optional[str],  # NEW: landcover path
    reference_grid: str,
    ca_boundary: str,
    out_root: str,
    compress: str = "lzw",
    skip_existing: bool = True  # Skip if aligned file already exists
) -> Tuple[Dict[str, str], Dict[str, List[str]], Optional[str]]:
    """
    Align all static, dynamic, and landcover rasters to the reference grid.
    
    Args:
        landcover_raster: Path to landcover.tif (optional)
        skip_existing: If True, skip alignment if output file already exists
    
    Returns:
        (aligned_static_paths, aligned_dynamic_paths, aligned_landcover_path)
    """
    print("\n[Aligning Static Rasters]")
    aligned_static = {}
    for var_name, src_path in static_rasters.items():
        out_path = os.path.join(out_root, "aligned_static", f"{var_name}_aligned.tif")
        
        if skip_existing and os.path.exists(out_path):
            print(f"  ✓ Using existing: {Path(out_path).name}")
            aligned_static[var_name] = out_path
        else:
            aligned_static[var_name] = align_raster_to_reference(
                src_path, reference_grid, out_path, ca_boundary, compress
            )
    
    # NEW: Align landcover
    aligned_landcover = None
    if landcover_raster and os.path.exists(landcover_raster):
        print("\n[Aligning Landcover Raster]")
        out_path = os.path.join(out_root, "aligned_static", "landcover_aligned.tif")
        
        if skip_existing and os.path.exists(out_path):
            print(f"  ✓ Using existing: {Path(out_path).name}")
            aligned_landcover = out_path
        else:
            aligned_landcover = align_raster_to_reference(
                landcover_raster, reference_grid, out_path, ca_boundary, compress
            )
    
    print("\n[Aligning Dynamic Rasters]")
    aligned_dynamic = {}
    for var_name, src_paths in dynamic_rasters.items():
        print(f"\n  Processing {var_name}: {len(src_paths)} files")
        aligned_dynamic[var_name] = []
        
        for src_path in tqdm(src_paths, desc=f"  Aligning {var_name}"):
            # Extract date from filename
            date_match = re.search(r'(\d{8})', Path(src_path).stem)
            date_str = date_match.group(1) if date_match else f"t{len(aligned_dynamic[var_name]):06d}"
            
            out_path = os.path.join(
                out_root, "aligned_dynamic", var_name, f"{var_name}_{date_str}_aligned.tif"
            )
            
            if skip_existing and os.path.exists(out_path):
                aligned_dynamic[var_name].append(out_path)
            else:
                aligned_dynamic[var_name].append(
                    align_raster_to_reference(src_path, reference_grid, out_path, ca_boundary, compress)
                )
    
    return aligned_static, aligned_dynamic, aligned_landcover


DATE8_RE = re.compile(r"(?<!\d)(\d{8})(?!\d)")

def parse_date_from_path(raster_path: str) -> Optional[datetime]:
    """Extract date from filename (YYYYMMDD format)"""
    m = DATE8_RE.search(Path(raster_path).stem)
    if m:
        try:
            return datetime.strptime(m.group(1), "%Y%m%d")
        except:
            pass
    return None


def one_hot_encode_landcover(landcover_patch: np.ndarray, num_classes: int = 10) -> np.ndarray:
    """
    Convert landcover patch to one-hot encoded format.
    
    Args:
        landcover_patch: 2D array with landcover classes (1-10)
        num_classes: Number of landcover classes (default: 10)
    
    Returns:
        3D array of shape (num_classes, height, width) with binary values
    """
    height, width = landcover_patch.shape
    one_hot = np.zeros((num_classes, height, width), dtype=np.float32)
    
    valid_mask = ~np.isnan(landcover_patch)
    
    for class_id in range(1, num_classes + 1):
        class_mask = (landcover_patch == class_id) & valid_mask
        one_hot[class_id - 1] = class_mask.astype(np.float32)
    
    return one_hot


def build_aligned_fire_dataset(
    aligned_static: Dict[str, str],  # {var_name: aligned_path}
    aligned_dynamic: Dict[str, List[str]],  # {var_name: [aligned_paths]}
    aligned_landcover: Optional[str],  # NEW: aligned landcover path
    fire_dataset: str,
    fire_date_field: str,
    ca_boundary: str,
    out_root: str = "output_aligned_fire_patches",
    tile_size: int = 25,
    window_days: int = 10,
    min_pos_frac: float = 0.80,
    min_valid_frac: float = 0.50,
    require_all_valid: bool = True,
    strict_inside: bool = True,
    num_landcover_classes: int = 10, 
    write_individual_static_tifs: bool = True,
    write_individual_dynamic_tifs_days: int = 2,
    write_individual_landcover_tifs: bool = True,
    tif_compress: str = "lzw",
    tif_nodata: float = -9999.0,
) -> pd.DataFrame:
    """
    Create aligned .npy patches for static, dynamic, and landcover data.
    
    NEW: Landcover is one-hot encoded to shape (num_classes, tile_size, tile_size)
    
    Output files:
      • Static NPY:     YYYYMMDD_row_col_static.npy
      • Dynamic NPY:    YYYYMMDD_row_col_dynamic.npy
      • Landcover NPY:  YYYYMMDD_row_col_clc_vec.npy  (one-hot encoded)
    """
    from rasterio.windows import transform as window_transform

    out_root = Path(out_root)
    out_root.mkdir(parents=True, exist_ok=True)

    # Reference from first static raster
    ref_path = next(iter(aligned_static.values()))
    with rasterio.open(ref_path) as ref:
        ref_crs = ref.crs
        ref_transform = ref.transform
        ref_height = ref.height
        ref_width = ref.width

    print(f"\n[Reference Grid] {ref_width}x{ref_height} px")

    # Boundary
    ca = gpd.read_file(ca_boundary)
    if ca.crs != ref_crs:
        ca = ca.to_crs(ref_crs)
    ca_union = ca.union_all()

    # Fires
    print("\n[Loading Fire Dataset]")
    fires = gpd.read_file(fire_dataset)
    if fires.crs != ref_crs:
        fires = fires.to_crs(ref_crs)
    if fire_date_field not in fires.columns:
        raise ValueError(f"Fire date field '{fire_date_field}' not found")

    def _pfd(v):
        if pd.isna(v): return None
        try: return pd.to_datetime(str(v)).date()
        except: return None

    fires['__fire_date__'] = fires[fire_date_field].apply(_pfd)
    fires = fires.dropna(subset=['__fire_date__'])
    fires_by_date = {d: grp for d, grp in fires.groupby('__fire_date__')}
    print(f"  ✓ Found {len(fires)} fire polygons across {len(fires_by_date)} unique dates")

    # Dynamic index
    print("\n[Indexing Dynamic Rasters]")
    dynamic_index = {}
    for var_name, paths in aligned_dynamic.items():
        dynamic_index[var_name] = {}
        for p in paths:
            dt = parse_date_from_path(p)
            if dt:
                dynamic_index[var_name][dt.date()] = p
        print(f"  {var_name}: {len(dynamic_index[var_name])} dates")

    # Date intersection across dynamic vars
    all_dates = set.intersection(*[set(d.keys()) for d in dynamic_index.values()])
    all_dates = sorted(all_dates)
    print(f"  ✓ {len(all_dates)} dates available across all dynamic variables")

    # Output dirs
    # static_dir = out_root / "static_patches"
    # dynamic_dir = out_root / "dynamic_patches"
    # landcover_dir = out_root / "landcover_patches"  # NEW
    # static_dir.mkdir(exist_ok=True)
    # dynamic_dir.mkdir(exist_ok=True)
    # landcover_dir.mkdir(exist_ok=True)
    npy_dir = Path(out_root) / "positive"
    npy_dir.mkdir(parents=True, exist_ok=True)

    # Individual per-variable GeoTIFF dirs
    static_tif_root = out_root / "static_tifs"
    dynamic_tif_root = out_root / "dynamic_tifs"
    landcover_tif_root = out_root / "landcover_tifs"  # NEW
    
    if write_individual_static_tifs:
        static_tif_root.mkdir(exist_ok=True)
    if write_individual_dynamic_tifs_days and write_individual_dynamic_tifs_days > 0:
        dynamic_tif_root.mkdir(exist_ok=True)
    if write_individual_landcover_tifs and aligned_landcover:
        landcover_tif_root.mkdir(exist_ok=True)

    manifest_rows = []
    patch_id = 0

    static_vars = sorted(aligned_static.keys())
    dynamic_vars = sorted(aligned_dynamic.keys())

    print(f"\n[Extracting Patches]")
    print(f"  Static variables: {static_vars}")
    print(f"  Dynamic variables: {dynamic_vars}")
    print(f"  Landcover classes: {num_landcover_classes if aligned_landcover else 'N/A'}")
    print(f"  Window days: {window_days}")

    for T in tqdm(all_dates, desc="Processing dates"):
        fires_T = fires_by_date.get(T)
        if fires_T is None or fires_T.empty:
            continue

        sindex = fires_T.sindex

        needed_dates = [T - timedelta(days=i) for i in range(window_days)]
        if not all(d in all_dates for d in needed_dates):
            continue

        rows = range(0, ref_height - (ref_height % tile_size), tile_size)
        cols = range(0, ref_width - (ref_width % tile_size), tile_size)

        for r0 in rows:
            for c0 in cols:
                win = Window(col_off=c0, row_off=r0, width=tile_size, height=tile_size)
                left, bottom, right, top = window_bounds(win, transform=ref_transform)
                tile_poly = box(left, bottom, right, top)

                if strict_inside and not ca_union.covers(tile_poly):
                    continue

                cand_idx = list(sindex.intersection(tile_poly.bounds))
                if not cand_idx:
                    continue

                inter_area = 0.0
                for geom in fires_T.geometry.iloc[cand_idx]:
                    if geom is None or geom.is_empty:
                        continue
                    try:
                        inter_area += geom.intersection(tile_poly).area
                    except Exception:
                        inter_area += geom.buffer(0).intersection(tile_poly).area

                tile_area = tile_poly.area
                overlap_frac = inter_area / tile_area if tile_area > 0 else 0.0
                if overlap_frac < min_pos_frac:
                    continue

                # --- Static stack ---
                static_stack = []
                valid_static = True
                for var_name in static_vars:
                    with rasterio.open(aligned_static[var_name]) as src:
                        patch = src.read(1, window=win, masked=True)
                        if patch.shape != (tile_size, tile_size):
                            valid_static = False; break
                        invalid = patch.mask
                        if require_all_valid and invalid.any():
                            valid_static = False; break
                        elif not require_all_valid:
                            valid_frac = 1.0 - float(invalid.mean())
                            if valid_frac < min_valid_frac:
                                valid_static = False; break
                        arr = np.asarray(patch, dtype=np.float32)
                        if invalid.any():
                            arr = np.where(invalid, np.nan, arr)
                        static_stack.append(arr)
                if not valid_static:
                    continue

                # --- Landcover (one-hot encoded) ---
                landcover_array = None
                if aligned_landcover:
                    with rasterio.open(aligned_landcover) as src:
                        lc_patch = src.read(1, window=win, masked=True)
                        if lc_patch.shape != (tile_size, tile_size):
                            continue
                        
                        # Convert to float and handle nodata
                        lc_arr = np.asarray(lc_patch, dtype=np.float32)
                        if lc_patch.mask.any():
                            lc_arr = np.where(lc_patch.mask, np.nan, lc_arr)
                        
                        # One-hot encode
                        landcover_array = one_hot_encode_landcover(lc_arr, num_landcover_classes)

                # --- Dynamic stack ---
                dynamic_stack = []
                valid_dynamic = True
                for date_offset in range(window_days):
                    target_date = T - timedelta(days=date_offset)
                    day_stack = []
                    for var_name in dynamic_vars:
                        raster_path = dynamic_index[var_name][target_date]
                        with rasterio.open(raster_path) as src:
                            patch = src.read(1, window=win, masked=True)
                            if patch.shape != (tile_size, tile_size):
                                valid_dynamic = False; break
                            invalid = patch.mask
                            if require_all_valid and invalid.any():
                                valid_dynamic = False; break
                            elif not require_all_valid:
                                valid_frac = 1.0 - float(invalid.mean())
                                if valid_frac < min_valid_frac:
                                    valid_dynamic = False; break
                            arr = np.asarray(patch, dtype=np.float32)
                            if invalid.any():
                                arr = np.where(invalid, np.nan, arr)
                            day_stack.append(arr)
                    if not valid_dynamic:
                        break
                    dynamic_stack.append(np.stack(day_stack, axis=0))
                if not valid_dynamic:
                    continue

                # --- Save NPYs ---
                patch_id += 1
                fire_day_str = T.strftime("%Y%m%d")
                static_array = np.stack(static_stack, axis=0)
                dynamic_array = np.stack(dynamic_stack, axis=0)

                static_path = npy_dir / f"{fire_day_str}_{int(r0)}_{int(c0)}_static.npy"
                dynamic_path = npy_dir / f"{fire_day_str}_{int(r0)}_{int(c0)}_dynamic.npy"
                np.save(static_path, static_array.astype(np.float32))
                np.save(dynamic_path, dynamic_array.astype(np.float32))
                
                # NEW: Save landcover as clc_vec.npy
                landcover_path = None
                if landcover_array is not None:
                    landcover_path = npy_dir / f"{fire_day_str}_{int(r0)}_{int(c0)}_clc_vec.npy"
                    np.save(landcover_path, landcover_array.astype(np.float32))

                # --- Write individual GeoTIFFs ---
                tile_transform = window_transform(win, ref_transform)

                # 1) Static TIFs
                if write_individual_static_tifs:
                    for idx, var_name in enumerate(static_vars):
                        arr = static_array[idx]
                        var_dir = (static_tif_root / var_name)
                        var_dir.mkdir(parents=True, exist_ok=True)
                        out_tif = var_dir / f"{fire_day_str}_{int(r0)}_{int(c0)}_static.tif"
                        profile = {
                            "driver": "GTiff",
                            "height": tile_size,
                            "width": tile_size,
                            "count": 1,
                            "dtype": "float32",
                            "crs": ref_crs,
                            "transform": tile_transform,
                            "nodata": tif_nodata,
                            "compress": tif_compress,
                        }
                        with rasterio.open(out_tif, "w", **profile) as dst:
                            dst.write(np.where(np.isnan(arr), tif_nodata, arr).astype(np.float32), 1)
                            dst.update_tags(
                                patch_id=int(patch_id),
                                fire_date=str(T),
                                row_off=int(r0),
                                col_off=int(c0),
                                var_name=var_name,
                            )

                # 2) Dynamic TIFs (first K days)
                K = max(0, min(int(write_individual_dynamic_tifs_days), window_days))
                for d in range(K):
                    day_date = (T - timedelta(days=d)).strftime("%Y%m%d")
                    for vidx, var_name in enumerate(dynamic_vars):
                        arr = dynamic_array[d, vidx]
                        var_dir = (dynamic_tif_root / var_name)
                        var_dir.mkdir(parents=True, exist_ok=True)
                        out_tif = var_dir / f"{day_date}_{int(r0)}_{int(c0)}_dynamic.tif"
                        profile = {
                            "driver": "GTiff",
                            "height": tile_size,
                            "width": tile_size,
                            "count": 1,
                            "dtype": "float32",
                            "crs": ref_crs,
                            "transform": tile_transform,
                            "nodata": tif_nodata,
                            "compress": tif_compress,
                        }
                        with rasterio.open(out_tif, "w", **profile) as dst:
                            dst.write(np.where(np.isnan(arr), tif_nodata, arr).astype(np.float32), 1)
                            dst.update_tags(
                                patch_id=int(patch_id),
                                fire_date=str(T),
                                day_index=int(d),
                                day_date=day_date,
                                row_off=int(r0),
                                col_off=int(c0),
                                var_name=var_name,
                            )

                # 3) NEW: Landcover TIFs (one per class)
                if write_individual_landcover_tifs and landcover_array is not None:
                    for class_id in range(num_landcover_classes):
                        arr = landcover_array[class_id]
                        class_name = f"class_{class_id + 1}"
                        var_dir = (landcover_tif_root / class_name)
                        var_dir.mkdir(parents=True, exist_ok=True)
                        out_tif = var_dir / f"{fire_day_str}_{int(r0)}_{int(c0)}_clc_vec.tif"
                        profile = {
                            "driver": "GTiff",
                            "height": tile_size,
                            "width": tile_size,
                            "count": 1,
                            "dtype": "float32",
                            "crs": ref_crs,
                            "transform": tile_transform,
                            "nodata": tif_nodata,
                            "compress": tif_compress,
                        }
                        with rasterio.open(out_tif, "w", **profile) as dst:
                            dst.write(arr.astype(np.float32), 1)
                            dst.update_tags(
                                patch_id=int(patch_id),
                                fire_date=str(T),
                                row_off=int(r0),
                                col_off=int(c0),
                                landcover_class=int(class_id + 1),
                            )

                # Manifest
                manifest_row = {
                    'patch_id': patch_id,
                    'static_path': str(static_path),
                    'dynamic_path': str(dynamic_path),
                    'fire_date': T.strftime('%Y-%m-%d'),
                    'row_off': r0,
                    'col_off': c0,
                    'left': left,
                    'bottom': bottom,
                    'right': right,
                    'top': top,
                    'overlap_frac': overlap_frac,
                    'static_vars': ','.join(static_vars),
                    'dynamic_vars': ','.join(dynamic_vars),
                    'window_days': window_days,
                }
                if landcover_path:
                    manifest_row['landcover_path'] = str(landcover_path)
                    manifest_row['landcover_classes'] = num_landcover_classes
                
                manifest_rows.append(manifest_row)

    # Save manifest
    manifest = pd.DataFrame(manifest_rows)
    manifest_path = out_root / "aligned_patches_manifest.csv"
    manifest.to_csv(manifest_path, index=False)

    print(f"\n✅ Complete!")
    print(f"  Total patches: {len(manifest)}")
    print(f"  Static shape: ({len(static_vars)}, {tile_size}, {tile_size})")
    print(f"  Dynamic shape: ({window_days}, {len(dynamic_vars)}, {tile_size}, {tile_size})")
    if aligned_landcover:
        print(f"  Landcover shape: ({num_landcover_classes}, {tile_size}, {tile_size}) [one-hot encoded]")
    print(f"  Manifest: {manifest_path}")
    print(f"  Static TIFs: {static_tif_root if write_individual_static_tifs else 'disabled'}")
    print(f"  Dynamic TIFs (first {write_individual_dynamic_tifs_days} days): "
          f"{dynamic_tif_root if write_individual_dynamic_tifs_days>0 else 'disabled'}")
    if aligned_landcover:
        print(f"  Landcover TIFs: {landcover_tif_root if write_individual_landcover_tifs else 'disabled'}")

    return manifest

In [4]:
config = {
    'ca_boundary': 'Dataset/Stataic Data/CA_State.gpkg',
    'fire_dataset': 'Dataset/Stataic Data/past_fire_2014_2024.gpkg',
    'fire_date_field': 'ALARM_DATE',
    
    'static_rasters': {
        'elevation': 'Dataset/Stataic Data/rasters_COP90/output_hh.tif',
        'slope': 'Dataset/Stataic Data/viz/Slope.tif',
        'population': 'Dataset/Stataic Data/Population/mosaic_masked.tif',
        'water_proximity': 'Dataset/Stataic Data/Waterway/ca_water_distance.tif',
        'road_proximity': 'Dataset/Stataic Data/roadways/ca_road_distance_snapped.tif',
    },
    'dynamic_folders': {
        'relative_humidity': 'Dataset/Dynamic Data/relative_humidity',
        'total_precipitation': 'Dataset/Dynamic Data/Precipitation',
    },
    'reference_grid_path': 'Dataset/Stataic Data/rasters_COP90/elevation_fixed_3310_1000m_clipped.tif',
    'out_root': 'output_aligned_patches',
    'tile_size': 4,
    'window_days': 10,  
    'dst_crs': 'EPSG:3310',
    'dst_res': 1000.0,  
    'skip_existing_aligned': True,  
    'min_pos_frac': 0.10,  
    'require_all_valid': False,
}

print("="*70)
print("ALIGNED FIRE DATASET BUILDER")
print("="*70)

print("\n[Step 0: Discovering Dynamic Rasters]")
dynamic_rasters = discover_dynamic_rasters(
    dynamic_folders=config['dynamic_folders'],
    pattern="*.tif"
)
print(f"\n✓ Total dynamic files found: {sum(len(files) for files in dynamic_rasters.values())}")

print("\n[Step 1: Creating Reference Grid]")
if os.path.exists(config['reference_grid_path']):
    print(f"  ✓ Using existing reference grid: {config['reference_grid_path']}")
    ref_grid = config['reference_grid_path']
else:
    ref_grid = create_reference_grid(
        ca_boundary=config['ca_boundary'],
        out_path=config['reference_grid_path'],
        dst_crs=config['dst_crs'],
        dst_res=config['dst_res']
    )

print("\n[Step 2: Aligning Rasters]")
print("⚠️  This may take a while with ~3,650 dynamic files (10 years × 365 days)")
print("    Set skip_existing_aligned=True to skip already processed files")

aligned_static, aligned_dynamic, aligned_landcover = align_all_rasters(
    static_rasters=config['static_rasters'],
    landcover_raster = "Dataset/Stataic Data/LandCover/land_cover_cal_reclass.tif",
    dynamic_rasters=dynamic_rasters,
    reference_grid=ref_grid,
    ca_boundary=config['ca_boundary'],
    out_root=config['out_root'],
    skip_existing=config['skip_existing_aligned']
)

print(f"\n✓ Alignment complete")
print(f"  Static: {len(aligned_static)} variables")
print(f"  Dynamic: {sum(len(files) for files in aligned_dynamic.values())} files across {len(aligned_dynamic)} variables")

print("\n[Step 3: Extracting Aligned Fire Patches]")
print("  This will create .npy files for patches that overlap with fires")

manifest = build_aligned_fire_dataset(
    aligned_static=aligned_static,
    aligned_dynamic=aligned_dynamic,
    aligned_landcover = aligned_landcover,
    fire_dataset=config['fire_dataset'],
    fire_date_field=config['fire_date_field'],
    ca_boundary=config['ca_boundary'],
    out_root=config['out_root'],
    tile_size=config['tile_size'],
    window_days=config['window_days'],
    min_pos_frac=config['min_pos_frac'],
    require_all_valid=config['require_all_valid']
)

print("\n" + "="*70)
print("📊 SUMMARY")
print("="*70)
print(f"Total fire patches extracted: {len(manifest)}")
print(f"\nStatic patch shape: ({len(aligned_static)}, {config['tile_size']}, {config['tile_size']})")
print(f"Dynamic patch shape: ({config['window_days']}, {len(aligned_dynamic)}, {config['tile_size']}, {config['tile_size']})")
print(f"\nOutput directory: {config['out_root']}")
print(f"Manifest file: {config['out_root']}/aligned_patches_manifest.csv")

if len(manifest) > 0:
    print(f"\n📝 First few patches:")
    print(manifest.head(10))
    
    print(f"\n📅 Date range:")
    print(f"  Earliest: {manifest['fire_date'].min()}")
    print(f"  Latest: {manifest['fire_date'].max()}")

print("\n" + "="*70)

ALIGNED FIRE DATASET BUILDER

[Step 0: Discovering Dynamic Rasters]
  relative_humidity: found 3656 files
  total_precipitation: found 3653 files

✓ Total dynamic files found: 7309

[Step 1: Creating Reference Grid]
  ✓ Using existing reference grid: Dataset/Stataic Data/rasters_COP90/elevation_fixed_3310_1000m_clipped.tif

[Step 2: Aligning Rasters]
⚠️  This may take a while with ~3,650 dynamic files (10 years × 365 days)
    Set skip_existing_aligned=True to skip already processed files

[Aligning Static Rasters]
  ✓ Using existing: elevation_aligned.tif
  ✓ Using existing: slope_aligned.tif
  ✓ Using existing: population_aligned.tif
  ✓ Using existing: water_proximity_aligned.tif
  ✓ Using existing: road_proximity_aligned.tif

[Aligning Landcover Raster]
  ✓ Using existing: landcover_aligned.tif

[Aligning Dynamic Rasters]

  Processing relative_humidity: 3656 files


  Aligning relative_humidity: 100%|██████| 3656/3656 [00:00<00:00, 97019.24it/s]



  Processing total_precipitation: 3653 files


  Aligning total_precipitation: 100%|████| 3653/3653 [00:00<00:00, 98383.11it/s]


✓ Alignment complete
  Static: 5 variables
  Dynamic: 7309 files across 2 variables

[Step 3: Extracting Aligned Fire Patches]
  This will create .npy files for patches that overlap with fires

[Reference Grid] 915x1056 px



/tmp/ipykernel_3808/510207535.py:322: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  ca_union = ca.unary_union



[Loading Fire Dataset]
  ✓ Found 4032 fire polygons across 1671 unique dates

[Indexing Dynamic Rasters]
  relative_humidity: 3653 dates
  total_precipitation: 3653 dates
  ✓ 3653 dates available across all dynamic variables

[Extracting Patches]
  Static variables: ['elevation', 'population', 'road_proximity', 'slope', 'water_proximity']
  Dynamic variables: ['relative_humidity', 'total_precipitation']
  Landcover classes: 10
  Window days: 10


Processing dates: 100%|███████████████████| 3653/3653 [1:27:23<00:00,  1.44s/it]


✅ Complete!
  Total patches: 4996
  Static shape: (5, 4, 4)
  Dynamic shape: (10, 2, 4, 4)
  Landcover shape: (10, 4, 4) [one-hot encoded]
  Manifest: output_aligned_patches/aligned_patches_manifest.csv
  Static TIFs: output_aligned_patches/static_tifs
  Dynamic TIFs (first 2 days): output_aligned_patches/dynamic_tifs
  Landcover TIFs: output_aligned_patches/landcover_tifs

📊 SUMMARY
Total fire patches extracted: 4996

Static patch shape: (5, 4, 4)
Dynamic patch shape: (10, 2, 4, 4)

Output directory: output_aligned_patches
Manifest file: output_aligned_patches/aligned_patches_manifest.csv

📝 First few patches:
   patch_id                                        static_path  \
0         1  output_aligned_patches/positive/20150206_424_4...   
1         2  output_aligned_patches/positive/20150207_504_4...   
2         3  output_aligned_patches/positive/20150207_508_4...   
3         4  output_aligned_patches/positive/20150207_508_4...   
4         5  output_aligned_patches/positive/20150

# Combining both Positive and Negative .npy file creation

In [2]:
import os
import re
import json
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Dict, Optional, Union, Tuple

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import box
import rasterio
from rasterio.windows import Window
from rasterio.windows import bounds as window_bounds
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.mask import mask
from tqdm import tqdm

def create_reference_grid(
    ca_boundary: str,
    out_path: str,
    dst_crs: str = "EPSG:3310",
    dst_res: float = 100.0,
    compress: str = "lzw"
) -> str:
    """
    Create a reference grid that covers California at specified resolution.
    This grid will be used to align ALL rasters (static and dynamic).
    
    Returns: path to the reference grid raster
    """
    print(f"[Creating Reference Grid]")
    ca = gpd.read_file(ca_boundary)
    if ca.crs is None:
        raise ValueError("CA boundary has no CRS")
    
    ca = ca.to_crs(dst_crs)
    bounds = ca.total_bounds  # minx, miny, maxx, maxy
    
    # Calculate dimensions
    width = int(np.ceil((bounds[2] - bounds[0]) / dst_res))
    height = int(np.ceil((bounds[3] - bounds[1]) / dst_res))
    
    # Create transform
    from rasterio.transform import from_origin
    transform = from_origin(bounds[0], bounds[3], dst_res, dst_res)
    
    # Create empty reference raster
    profile = {
        'driver': 'GTiff',
        'height': height,
        'width': width,
        'count': 1,
        'dtype': 'float32',
        'crs': dst_crs,
        'transform': transform,
        'nodata': -9999.0,
        'compress': compress
    }
    
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with rasterio.open(out_path, 'w', **profile) as dst:
        dst.write(np.zeros((height, width), dtype='float32'), 1)
    
    print(f"  ✓ Reference grid created: {width}x{height} px at {dst_res}m resolution")
    print(f"  ✓ Saved to: {out_path}")
    return out_path


def align_raster_to_reference(
    src_raster: str,
    reference_grid: str,
    out_path: str,
    ca_boundary: str,
    compress: str = "lzw"
) -> str:
    """
    Reproject and snap source raster to reference grid, then clip to CA boundary.
    """
    print(f"  Processing: {Path(src_raster).name}")
    
    # Convert to Path object and create parent directories
    out_path = Path(out_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    
    # Step 1: Reproject to match reference grid
    with rasterio.open(reference_grid) as ref, rasterio.open(src_raster) as src:
        profile = ref.profile.copy()
        profile.update({'compress': compress})
        
        temp_path = str(out_path.parent / (out_path.name + ".temp.tif"))
        with rasterio.open(temp_path, 'w', **profile) as dst:
            reproject(
                source=rasterio.band(src, 1),
                destination=rasterio.band(dst, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=ref.transform,
                dst_crs=ref.crs,
                resampling=Resampling.nearest,  # Use nearest for categorical data
                src_nodata=src.nodata,
                dst_nodata=ref.nodata
            )
    
    # Step 2: Clip to CA boundary
    ca = gpd.read_file(ca_boundary)
    with rasterio.open(reference_grid) as ref:
        if ca.crs != ref.crs:
            ca = ca.to_crs(ref.crs)
    
    geoms = [geom.__geo_interface__ for geom in ca.geometry if geom is not None]
    
    with rasterio.open(temp_path) as src:
        out_image, out_transform = mask(src, geoms, crop=True, nodata=src.nodata, filled=True)
        out_meta = src.meta.copy()
        out_meta.update({
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform,
            "compress": compress
        })
        
        with rasterio.open(str(out_path), "w", **out_meta) as dst:
            dst.write(out_image)
    
    # Clean up temp file
    if os.path.exists(temp_path):
        os.remove(temp_path)
    
    print(f"    ✓ Aligned and clipped: {out_path.name}")
    return str(out_path)


def discover_dynamic_rasters(
    dynamic_folders: Dict[str, str],
    pattern: str = "*.tif"
) -> Dict[str, List[str]]:
    """
    Discover all .tif files in each dynamic variable folder.
    
    Args:
        dynamic_folders: {variable_name: folder_path containing all tifs for that variable}
        pattern: file pattern to match (default: "*.tif")
    
    Returns:
        {variable_name: [sorted list of paths]}
    """
    dynamic_rasters = {}
    for var_name, folder_path in dynamic_folders.items():
        folder = Path(folder_path)
        if not folder.exists():
            raise ValueError(f"Folder not found: {folder_path}")
        
        tif_files = sorted(folder.glob(pattern))
        if not tif_files:
            raise ValueError(f"No .tif files found in {folder_path}")
        
        dynamic_rasters[var_name] = [str(p) for p in tif_files]
        print(f"  {var_name}: found {len(tif_files)} files")
    
    return dynamic_rasters


def align_all_rasters(
    static_rasters: Dict[str, str],  # {variable_name: path}
    dynamic_rasters: Dict[str, List[str]],  # {variable_name: [path1, path2, ...]}
    landcover_raster: Optional[str],  # NEW: landcover path
    reference_grid: str,
    ca_boundary: str,
    out_root: str,
    compress: str = "lzw",
    skip_existing: bool = True  # Skip if aligned file already exists
) -> Tuple[Dict[str, str], Dict[str, List[str]], Optional[str]]:
    """
    Align all static, dynamic, and landcover rasters to the reference grid.
    
    Args:
        landcover_raster: Path to landcover.tif (optional)
        skip_existing: If True, skip alignment if output file already exists
    
    Returns:
        (aligned_static_paths, aligned_dynamic_paths, aligned_landcover_path)
    """
    print("\n[Aligning Static Rasters]")
    aligned_static = {}
    for var_name, src_path in static_rasters.items():
        out_path = os.path.join(out_root, "aligned_static", f"{var_name}_aligned.tif")
        
        if skip_existing and os.path.exists(out_path):
            print(f"  ✓ Using existing: {Path(out_path).name}")
            aligned_static[var_name] = out_path
        else:
            aligned_static[var_name] = align_raster_to_reference(
                src_path, reference_grid, out_path, ca_boundary, compress
            )
    
    # NEW: Align landcover
    aligned_landcover = None
    if landcover_raster and os.path.exists(landcover_raster):
        print("\n[Aligning Landcover Raster]")
        out_path = os.path.join(out_root, "aligned_static", "landcover_aligned.tif")
        
        if skip_existing and os.path.exists(out_path):
            print(f"  ✓ Using existing: {Path(out_path).name}")
            aligned_landcover = out_path
        else:
            aligned_landcover = align_raster_to_reference(
                landcover_raster, reference_grid, out_path, ca_boundary, compress
            )
    
    print("\n[Aligning Dynamic Rasters]")
    aligned_dynamic = {}
    for var_name, src_paths in dynamic_rasters.items():
        print(f"\n  Processing {var_name}: {len(src_paths)} files")
        aligned_dynamic[var_name] = []
        
        for src_path in tqdm(src_paths, desc=f"  Aligning {var_name}"):
            # Extract date from filename
            date_match = re.search(r'(\d{8})', Path(src_path).stem)
            date_str = date_match.group(1) if date_match else f"t{len(aligned_dynamic[var_name]):06d}"
            
            out_path = os.path.join(
                out_root, "aligned_dynamic", var_name, f"{var_name}_{date_str}_aligned.tif"
            )
            
            if skip_existing and os.path.exists(out_path):
                aligned_dynamic[var_name].append(out_path)
            else:
                aligned_dynamic[var_name].append(
                    align_raster_to_reference(src_path, reference_grid, out_path, ca_boundary, compress)
                )
    
    return aligned_static, aligned_dynamic, aligned_landcover


DATE8_RE = re.compile(r"(?<!\d)(\d{8})(?!\d)")

def parse_date_from_path(raster_path: str) -> Optional[datetime]:
    """Extract date from filename (YYYYMMDD format)"""
    m = DATE8_RE.search(Path(raster_path).stem)
    if m:
        try:
            return datetime.strptime(m.group(1), "%Y%m%d")
        except:
            pass
    return None


def one_hot_encode_landcover(landcover_patch: np.ndarray, num_classes: int = 10) -> np.ndarray:
    """
    Convert landcover patch to one-hot encoded format.
    
    Args:
        landcover_patch: 2D array with landcover classes (1-10)
        num_classes: Number of landcover classes (default: 10)
    
    Returns:
        3D array of shape (num_classes, height, width) with binary values
    """
    height, width = landcover_patch.shape
    one_hot = np.zeros((num_classes, height, width), dtype=np.float32)
    
    valid_mask = ~np.isnan(landcover_patch)
    
    for class_id in range(1, num_classes + 1):
        class_mask = (landcover_patch == class_id) & valid_mask
        one_hot[class_id - 1] = class_mask.astype(np.float32)
    
    return one_hot


def build_aligned_fire_dataset(
    aligned_static: Dict[str, str],  
    aligned_dynamic: Dict[str, List[str]], 
    aligned_landcover: Optional[str],  
    fire_dataset: str,
    fire_date_field: str,
    ca_boundary: str,
    out_root: str = "output_aligned_fire_patches",
    tile_size: int = 25,
    window_days: int = 10,
    min_pos_frac: float = 0.80,
    min_valid_frac: float = 0.50,
    require_all_valid: bool = True,
    strict_inside: bool = True,
    num_landcover_classes: int = 10,  # NEW: number of landcover classes
    # Controls for individual TIFs
    write_individual_static_tifs: bool = True,
    write_individual_dynamic_tifs_days: int = 2,
    write_individual_landcover_tifs: bool = True,  # NEW
    tif_compress: str = "lzw",
    tif_nodata: float = -9999.0,
) -> pd.DataFrame:
    """
    Create aligned .npy patches for static, dynamic, and landcover data.
    
    NEW: Landcover is one-hot encoded to shape (num_classes, tile_size, tile_size)
    
    Output files:
      • Static NPY:     YYYYMMDD_row_col_static.npy
      • Dynamic NPY:    YYYYMMDD_row_col_dynamic.npy
      • Landcover NPY:  YYYYMMDD_row_col_clc_vec.npy  (one-hot encoded)
    """
    from rasterio.windows import transform as window_transform

    out_root = Path(out_root)
    out_root.mkdir(parents=True, exist_ok=True)

    # Reference from first static raster
    ref_path = next(iter(aligned_static.values()))
    with rasterio.open(ref_path) as ref:
        ref_crs = ref.crs
        ref_transform = ref.transform
        ref_height = ref.height
        ref_width = ref.width

    print(f"\n[Reference Grid] {ref_width}x{ref_height} px")

    # Boundary
    ca = gpd.read_file(ca_boundary)
    if ca.crs != ref_crs:
        ca = ca.to_crs(ref_crs)
    ca_union = ca.unary_union

    # Fires
    print("\n[Loading Fire Dataset]")
    fires = gpd.read_file(fire_dataset)
    if fires.crs != ref_crs:
        fires = fires.to_crs(ref_crs)
    if fire_date_field not in fires.columns:
        raise ValueError(f"Fire date field '{fire_date_field}' not found")

    def _pfd(v):
        if pd.isna(v): return None
        try: return pd.to_datetime(str(v)).date()
        except: return None

    fires['__fire_date__'] = fires[fire_date_field].apply(_pfd)
    fires = fires.dropna(subset=['__fire_date__'])
    fires_by_date = {d: grp for d, grp in fires.groupby('__fire_date__')}
    print(f"  ✓ Found {len(fires)} fire polygons across {len(fires_by_date)} unique dates")

    # Dynamic index
    print("\n[Indexing Dynamic Rasters]")
    dynamic_index = {}
    for var_name, paths in aligned_dynamic.items():
        dynamic_index[var_name] = {}
        for p in paths:
            dt = parse_date_from_path(p)
            if dt:
                dynamic_index[var_name][dt.date()] = p
        print(f"  {var_name}: {len(dynamic_index[var_name])} dates")

    # Date intersection across dynamic vars
    all_dates = set.intersection(*[set(d.keys()) for d in dynamic_index.values()])
    all_dates = sorted(all_dates)
    print(f"  ✓ {len(all_dates)} dates available across all dynamic variables")

    # Output dirs
    # static_dir = out_root / "static_patches"
    # dynamic_dir = out_root / "dynamic_patches"
    # landcover_dir = out_root / "landcover_patches"  # NEW
    # static_dir.mkdir(exist_ok=True)
    # dynamic_dir.mkdir(exist_ok=True)
    # landcover_dir.mkdir(exist_ok=True)
    npy_dir = Path(out_root) / "positive"
    npy_dir.mkdir(parents=True, exist_ok=True)

    # Individual per-variable GeoTIFF dirs
    static_tif_root = out_root / "static_tifs"
    dynamic_tif_root = out_root / "dynamic_tifs"
    landcover_tif_root = out_root / "landcover_tifs"  # NEW
    
    if write_individual_static_tifs:
        static_tif_root.mkdir(exist_ok=True)
    if write_individual_dynamic_tifs_days and write_individual_dynamic_tifs_days > 0:
        dynamic_tif_root.mkdir(exist_ok=True)
    if write_individual_landcover_tifs and aligned_landcover:
        landcover_tif_root.mkdir(exist_ok=True)

    manifest_rows = []
    patch_id = 0

    static_vars = sorted(aligned_static.keys())
    dynamic_vars = sorted(aligned_dynamic.keys())

    print(f"\n[Extracting Patches]")
    print(f"  Static variables: {static_vars}")
    print(f"  Dynamic variables: {dynamic_vars}")
    print(f"  Landcover classes: {num_landcover_classes if aligned_landcover else 'N/A'}")
    print(f"  Window days: {window_days}")

    for T in tqdm(all_dates, desc="Processing dates"):
        fires_T = fires_by_date.get(T)
        if fires_T is None or fires_T.empty:
            continue

        sindex = fires_T.sindex

        needed_dates = [T - timedelta(days=i) for i in range(window_days)]
        if not all(d in all_dates for d in needed_dates):
            continue

        rows = range(0, ref_height - (ref_height % tile_size), tile_size)
        cols = range(0, ref_width - (ref_width % tile_size), tile_size)

        for r0 in rows:
            for c0 in cols:
                win = Window(col_off=c0, row_off=r0, width=tile_size, height=tile_size)
                left, bottom, right, top = window_bounds(win, transform=ref_transform)
                tile_poly = box(left, bottom, right, top)

                if strict_inside and not ca_union.covers(tile_poly):
                    continue

                cand_idx = list(sindex.intersection(tile_poly.bounds))
                if not cand_idx:
                    continue

                inter_area = 0.0
                for geom in fires_T.geometry.iloc[cand_idx]:
                    if geom is None or geom.is_empty:
                        continue
                    try:
                        inter_area += geom.intersection(tile_poly).area
                    except Exception:
                        inter_area += geom.buffer(0).intersection(tile_poly).area

                tile_area = tile_poly.area
                overlap_frac = inter_area / tile_area if tile_area > 0 else 0.0
                if overlap_frac < min_pos_frac:
                    continue

                # --- Static stack ---
                static_stack = []
                valid_static = True
                for var_name in static_vars:
                    with rasterio.open(aligned_static[var_name]) as src:
                        patch = src.read(1, window=win, masked=True)
                        if patch.shape != (tile_size, tile_size):
                            valid_static = False; break
                        invalid = patch.mask
                        if require_all_valid and invalid.any():
                            valid_static = False; break
                        elif not require_all_valid:
                            valid_frac = 1.0 - float(invalid.mean())
                            if valid_frac < min_valid_frac:
                                valid_static = False; break
                        arr = np.asarray(patch, dtype=np.float32)
                        if invalid.any():
                            arr = np.where(invalid, np.nan, arr)
                        static_stack.append(arr)
                if not valid_static:
                    continue

                # --- Landcover (one-hot encoded) ---
                landcover_array = None
                if aligned_landcover:
                    with rasterio.open(aligned_landcover) as src:
                        lc_patch = src.read(1, window=win, masked=True)
                        if lc_patch.shape != (tile_size, tile_size):
                            continue
                        
                        # Convert to float and handle nodata
                        lc_arr = np.asarray(lc_patch, dtype=np.float32)
                        if lc_patch.mask.any():
                            lc_arr = np.where(lc_patch.mask, np.nan, lc_arr)
                        
                        # One-hot encode
                        landcover_array = one_hot_encode_landcover(lc_arr, num_landcover_classes)

                # --- Dynamic stack ---
                dynamic_stack = []
                valid_dynamic = True
                for date_offset in range(window_days):
                    target_date = T - timedelta(days=date_offset)
                    day_stack = []
                    for var_name in dynamic_vars:
                        raster_path = dynamic_index[var_name][target_date]
                        with rasterio.open(raster_path) as src:
                            patch = src.read(1, window=win, masked=True)
                            if patch.shape != (tile_size, tile_size):
                                valid_dynamic = False; break
                            invalid = patch.mask
                            if require_all_valid and invalid.any():
                                valid_dynamic = False; break
                            elif not require_all_valid:
                                valid_frac = 1.0 - float(invalid.mean())
                                if valid_frac < min_valid_frac:
                                    valid_dynamic = False; break
                            arr = np.asarray(patch, dtype=np.float32)
                            if invalid.any():
                                arr = np.where(invalid, np.nan, arr)
                            day_stack.append(arr)
                    if not valid_dynamic:
                        break
                    dynamic_stack.append(np.stack(day_stack, axis=0))
                if not valid_dynamic:
                    continue

                # --- Save NPYs ---
                patch_id += 1
                fire_day_str = T.strftime("%Y%m%d")
                static_array = np.stack(static_stack, axis=0)
                dynamic_array = np.stack(dynamic_stack, axis=0)

                static_path = npy_dir / f"{fire_day_str}_{int(r0)}_{int(c0)}_static.npy"
                dynamic_path = npy_dir / f"{fire_day_str}_{int(r0)}_{int(c0)}_dynamic.npy"
                np.save(static_path, static_array.astype(np.float32))
                np.save(dynamic_path, dynamic_array.astype(np.float32))
                
                # NEW: Save landcover as clc_vec.npy
                landcover_path = None
                if landcover_array is not None:
                    landcover_path = npy_dir / f"{fire_day_str}_{int(r0)}_{int(c0)}_clc_vec.npy"
                    np.save(landcover_path, landcover_array.astype(np.float32))

                # --- Write individual GeoTIFFs ---
                tile_transform = window_transform(win, ref_transform)

                # 1) Static TIFs
                if write_individual_static_tifs:
                    for idx, var_name in enumerate(static_vars):
                        arr = static_array[idx]
                        var_dir = (static_tif_root / var_name)
                        var_dir.mkdir(parents=True, exist_ok=True)
                        out_tif = var_dir / f"{fire_day_str}_{int(r0)}_{int(c0)}_static.tif"
                        profile = {
                            "driver": "GTiff",
                            "height": tile_size,
                            "width": tile_size,
                            "count": 1,
                            "dtype": "float32",
                            "crs": ref_crs,
                            "transform": tile_transform,
                            "nodata": tif_nodata,
                            "compress": tif_compress,
                        }
                        with rasterio.open(out_tif, "w", **profile) as dst:
                            dst.write(np.where(np.isnan(arr), tif_nodata, arr).astype(np.float32), 1)
                            dst.update_tags(
                                patch_id=int(patch_id),
                                fire_date=str(T),
                                row_off=int(r0),
                                col_off=int(c0),
                                var_name=var_name,
                            )

                # 2) Dynamic TIFs (first K days)
                K = max(0, min(int(write_individual_dynamic_tifs_days), window_days))
                for d in range(K):
                    day_date = (T - timedelta(days=d)).strftime("%Y%m%d")
                    for vidx, var_name in enumerate(dynamic_vars):
                        arr = dynamic_array[d, vidx]
                        var_dir = (dynamic_tif_root / var_name)
                        var_dir.mkdir(parents=True, exist_ok=True)
                        out_tif = var_dir / f"{day_date}_{int(r0)}_{int(c0)}_dynamic.tif"
                        profile = {
                            "driver": "GTiff",
                            "height": tile_size,
                            "width": tile_size,
                            "count": 1,
                            "dtype": "float32",
                            "crs": ref_crs,
                            "transform": tile_transform,
                            "nodata": tif_nodata,
                            "compress": tif_compress,
                        }
                        with rasterio.open(out_tif, "w", **profile) as dst:
                            dst.write(np.where(np.isnan(arr), tif_nodata, arr).astype(np.float32), 1)
                            dst.update_tags(
                                patch_id=int(patch_id),
                                fire_date=str(T),
                                day_index=int(d),
                                day_date=day_date,
                                row_off=int(r0),
                                col_off=int(c0),
                                var_name=var_name,
                            )

                # 3) NEW: Landcover TIFs (one per class)
                if write_individual_landcover_tifs and landcover_array is not None:
                    for class_id in range(num_landcover_classes):
                        arr = landcover_array[class_id]
                        class_name = f"class_{class_id + 1}"
                        var_dir = (landcover_tif_root / class_name)
                        var_dir.mkdir(parents=True, exist_ok=True)
                        out_tif = var_dir / f"{fire_day_str}_{int(r0)}_{int(c0)}_clc_vec.tif"
                        profile = {
                            "driver": "GTiff",
                            "height": tile_size,
                            "width": tile_size,
                            "count": 1,
                            "dtype": "float32",
                            "crs": ref_crs,
                            "transform": tile_transform,
                            "nodata": tif_nodata,
                            "compress": tif_compress,
                        }
                        with rasterio.open(out_tif, "w", **profile) as dst:
                            dst.write(arr.astype(np.float32), 1)
                            dst.update_tags(
                                patch_id=int(patch_id),
                                fire_date=str(T),
                                row_off=int(r0),
                                col_off=int(c0),
                                landcover_class=int(class_id + 1),
                            )

                # Manifest
                manifest_row = {
                    'patch_id': patch_id,
                    'static_path': str(static_path),
                    'dynamic_path': str(dynamic_path),
                    'fire_date': T.strftime('%Y-%m-%d'),
                    'row_off': r0,
                    'col_off': c0,
                    'left': left,
                    'bottom': bottom,
                    'right': right,
                    'top': top,
                    'overlap_frac': overlap_frac,
                    'static_vars': ','.join(static_vars),
                    'dynamic_vars': ','.join(dynamic_vars),
                    'window_days': window_days,
                }
                if landcover_path:
                    manifest_row['landcover_path'] = str(landcover_path)
                    manifest_row['landcover_classes'] = num_landcover_classes
                
                manifest_rows.append(manifest_row)

    # Save manifest
    manifest = pd.DataFrame(manifest_rows)
    manifest_path = out_root / "aligned_patches_manifest.csv"
    manifest.to_csv(manifest_path, index=False)

    print(f"\n✅ Complete!")
    print(f"  Total patches: {len(manifest)}")
    print(f"  Static shape: ({len(static_vars)}, {tile_size}, {tile_size})")
    print(f"  Dynamic shape: ({window_days}, {len(dynamic_vars)}, {tile_size}, {tile_size})")
    if aligned_landcover:
        print(f"  Landcover shape: ({num_landcover_classes}, {tile_size}, {tile_size}) [one-hot encoded]")
    print(f"  Manifest: {manifest_path}")
    print(f"  Static TIFs: {static_tif_root if write_individual_static_tifs else 'disabled'}")
    print(f"  Dynamic TIFs (first {write_individual_dynamic_tifs_days} days): "
          f"{dynamic_tif_root if write_individual_dynamic_tifs_days>0 else 'disabled'}")
    if aligned_landcover:
        print(f"  Landcover TIFs: {landcover_tif_root if write_individual_landcover_tifs else 'disabled'}")

    return manifest


def build_negative_fire_dataset(
    aligned_static: Dict[str, str],
    aligned_dynamic: Dict[str, List[str]],
    aligned_landcover: Optional[str],
    fire_dataset: str,
    fire_date_field: str,
    ca_boundary: str,
    out_root: str = "output_aligned_fire_patches",
    tile_size: int = 25,
    window_days: int = 10,
    samples_per_year: int = 1000,
    min_valid_frac: float = 0.50,
    require_all_valid: bool = True,
    num_landcover_classes: int = 10,
    start_year: int = 2015,
    end_year: int = 2024,
    max_attempts_per_year: int = 50000,
    tif_nodata: float = -9999.0,
) -> pd.DataFrame:
    """
    Create negative samples where NO fire occurred in the 10-day window at sampled locations.
    
    Args:
        samples_per_year: Number of negative samples to generate per year (default: 3000)
        start_year: First year to sample from (default: 2015)
        end_year: Last year to sample from (default: 2024)
        max_attempts_per_year: Maximum random sampling attempts per year before giving up
    
    Returns:
        DataFrame with manifest of negative samples
    """
    from rasterio.windows import transform as window_transform
    import random

    out_root = Path(out_root)
    
    # Reference from first static raster
    ref_path = next(iter(aligned_static.values()))
    with rasterio.open(ref_path) as ref:
        ref_crs = ref.crs
        ref_transform = ref.transform
        ref_height = ref.height
        ref_width = ref.width

    print(f"\n[Reference Grid] {ref_width}x{ref_height} px")

    # Boundary
    ca = gpd.read_file(ca_boundary)
    if ca.crs != ref_crs:
        ca = ca.to_crs(ref_crs)
    ca_union = ca.unary_union

    # Fires
    print("\n[Loading Fire Dataset]")
    fires = gpd.read_file(fire_dataset)
    if fires.crs != ref_crs:
        fires = fires.to_crs(ref_crs)
    if fire_date_field not in fires.columns:
        raise ValueError(f"Fire date field '{fire_date_field}' not found")

    def _pfd(v):
        if pd.isna(v): return None
        try: return pd.to_datetime(str(v)).date()
        except: return None

    fires['__fire_date__'] = fires[fire_date_field].apply(_pfd)
    fires = fires.dropna(subset=['__fire_date__'])
    fires_by_date = {d: grp for d, grp in fires.groupby('__fire_date__')}
    print(f"  ✓ Found {len(fires)} fire polygons across {len(fires_by_date)} unique dates")

    # Dynamic index
    print("\n[Indexing Dynamic Rasters]")
    dynamic_index = {}
    for var_name, paths in aligned_dynamic.items():
        dynamic_index[var_name] = {}
        for p in paths:
            dt = parse_date_from_path(p)
            if dt:
                dynamic_index[var_name][dt.date()] = p
        print(f"  {var_name}: {len(dynamic_index[var_name])} dates")

    # Date intersection across dynamic vars
    all_dates = set.intersection(*[set(d.keys()) for d in dynamic_index.values()])
    all_dates = sorted(all_dates)
    print(f"  ✓ {len(all_dates)} dates available across all dynamic variables")

    negative_root = out_root / "negative"
    negative_root.mkdir(parents=True, exist_ok=True)

    manifest_rows = []
    patch_id = 0

    static_vars = sorted(aligned_static.keys())
    dynamic_vars = sorted(aligned_dynamic.keys())

    print(f"\n[Extracting NEGATIVE Patches]")
    print(f"  Static variables: {static_vars}")
    print(f"  Dynamic variables: {dynamic_vars}")
    print(f"  Landcover classes: {num_landcover_classes if aligned_landcover else 'N/A'}")
    print(f"  Window days: {window_days}")
    print(f"  Samples per year: {samples_per_year}")
    print(f"  Year range: {start_year}-{end_year}")

    # Pre-compute valid tile positions (inside CA boundary)
    print("\n[Pre-computing valid tile positions]")
    max_row = ref_height - tile_size
    max_col = ref_width - tile_size
    
    # Generate all possible tile positions
    all_tile_positions = []
    for r0 in range(0, max_row, tile_size):
        for c0 in range(0, max_col, tile_size):
            win = Window(col_off=c0, row_off=r0, width=tile_size, height=tile_size)
            left, bottom, right, top = window_bounds(win, transform=ref_transform)
            tile_poly = box(left, bottom, right, top)
            
            # Only keep tiles that are inside CA
            if ca_union.covers(tile_poly):
                all_tile_positions.append((r0, c0, tile_poly, left, bottom, right, top))
    
    print(f"  ✓ Found {len(all_tile_positions)} valid tile positions inside California")

    # Process each year
    for year in range(start_year, end_year + 1):
        print(f"\n[Processing Year {year}]")
        
        # Get dates for this year
        year_dates = [d for d in all_dates if d.year == year]
        
        # Filter dates that have 10-day window available
        valid_dates = []
        for T in year_dates:
            needed_dates = [T - timedelta(days=i) for i in range(window_days)]
            if all(d in all_dates for d in needed_dates):
                valid_dates.append(T)
        
        print(f"  Valid dates with 10-day window: {len(valid_dates)}")
        
        if len(valid_dates) == 0:
            print(f"  ⚠️  No valid dates found for year {year}, skipping")
            continue
        
        year_samples = 0
        attempts = 0
        
        with tqdm(total=samples_per_year, desc=f"  Sampling year {year}") as pbar:
            while year_samples < samples_per_year and attempts < max_attempts_per_year:
                attempts += 1
                
                # Randomly select a date
                T = random.choice(valid_dates)
                
                # Get 10-day window
                needed_dates = [T - timedelta(days=i) for i in range(window_days)]
                
                # Check if any fire occurred in this 10-day window
                fires_in_window = []
                has_fire_in_window = False
                for date in needed_dates:
                    if date in fires_by_date:
                        fires_in_window.extend(fires_by_date[date].geometry.tolist())
                        has_fire_in_window = True
                
                # Create spatial index for fires in this window (if any)
                if has_fire_in_window:
                    fire_union = gpd.GeoSeries(fires_in_window, crs=ref_crs).union_all()
                else:
                    fire_union = None
                
                # Randomly select a tile position
                r0, c0, tile_poly, left, bottom, right, top = random.choice(all_tile_positions)
                
                # Check if this tile intersects with any fire in the 10-day window
                if fire_union is not None:
                    try:
                        if tile_poly.intersects(fire_union):
                            continue  # Skip this tile, has fire
                    except Exception:
                        # Handle geometry errors
                        if tile_poly.intersects(fire_union.buffer(0)):
                            continue
                
                # This tile has NO fire in the 10-day window, extract data
                win = Window(col_off=c0, row_off=r0, width=tile_size, height=tile_size)
                
                # --- Static stack ---
                static_stack = []
                valid_static = True
                for var_name in static_vars:
                    with rasterio.open(aligned_static[var_name]) as src:
                        patch = src.read(1, window=win, masked=True)
                        if patch.shape != (tile_size, tile_size):
                            valid_static = False
                            break
                        invalid = patch.mask
                        if require_all_valid and invalid.any():
                            valid_static = False
                            break
                        elif not require_all_valid:
                            valid_frac = 1.0 - float(invalid.mean())
                            if valid_frac < min_valid_frac:
                                valid_static = False
                                break
                        arr = np.asarray(patch, dtype=np.float32)
                        if invalid.any():
                            arr = np.where(invalid, np.nan, arr)
                        static_stack.append(arr)
                if not valid_static:
                    continue

                # --- Landcover (one-hot encoded) ---
                landcover_array = None
                if aligned_landcover:
                    with rasterio.open(aligned_landcover) as src:
                        lc_patch = src.read(1, window=win, masked=True)
                        if lc_patch.shape != (tile_size, tile_size):
                            continue
                        
                        lc_arr = np.asarray(lc_patch, dtype=np.float32)
                        if lc_patch.mask.any():
                            lc_arr = np.where(lc_patch.mask, np.nan, lc_arr)
                        
                        landcover_array = one_hot_encode_landcover(lc_arr, num_landcover_classes)

                # --- Dynamic stack ---
                dynamic_stack = []
                valid_dynamic = True
                for date_offset in range(window_days):
                    target_date = T - timedelta(days=date_offset)
                    day_stack = []
                    for var_name in dynamic_vars:
                        raster_path = dynamic_index[var_name][target_date]
                        with rasterio.open(raster_path) as src:
                            patch = src.read(1, window=win, masked=True)
                            if patch.shape != (tile_size, tile_size):
                                valid_dynamic = False
                                break
                            invalid = patch.mask
                            if require_all_valid and invalid.any():
                                valid_dynamic = False
                                break
                            elif not require_all_valid:
                                valid_frac = 1.0 - float(invalid.mean())
                                if valid_frac < min_valid_frac:
                                    valid_dynamic = False
                                    break
                            arr = np.asarray(patch, dtype=np.float32)
                            if invalid.any():
                                arr = np.where(invalid, np.nan, arr)
                            day_stack.append(arr)
                    if not valid_dynamic:
                        break
                    dynamic_stack.append(np.stack(day_stack, axis=0))
                if not valid_dynamic:
                    continue

                # --- Save NPYs ---
                patch_id += 1
                year_samples += 1
                pbar.update(1)
                
                fire_day_str = T.strftime("%Y%m%d")
                static_array = np.stack(static_stack, axis=0)
                dynamic_array = np.stack(dynamic_stack, axis=0)

                static_path = negative_root / f"{fire_day_str}_{int(r0)}_{int(c0)}_static.npy"
                dynamic_path = negative_root / f"{fire_day_str}_{int(r0)}_{int(c0)}_dynamic.npy"
                np.save(static_path, static_array.astype(np.float32))
                np.save(dynamic_path, dynamic_array.astype(np.float32))
                
                landcover_path = None
                if landcover_array is not None:
                    landcover_path = negative_root / f"{fire_day_str}_{int(r0)}_{int(c0)}_clc_vec.npy"
                    np.save(landcover_path, landcover_array.astype(np.float32))

                # Manifest
                manifest_row = {
                    'patch_id': patch_id,
                    'static_path': str(static_path),
                    'dynamic_path': str(dynamic_path),
                    'sample_date': T.strftime('%Y-%m-%d'),
                    'year': year,
                    'row_off': r0,
                    'col_off': c0,
                    'left': left,
                    'bottom': bottom,
                    'right': right,
                    'top': top,
                    'label': 0,  # Negative sample
                    'static_vars': ','.join(static_vars),
                    'dynamic_vars': ','.join(dynamic_vars),
                    'window_days': window_days,
                }
                if landcover_path:
                    manifest_row['landcover_path'] = str(landcover_path)
                    manifest_row['landcover_classes'] = num_landcover_classes
                
                manifest_rows.append(manifest_row)
        
        print(f"  ✓ Year {year}: Generated {year_samples} negative samples (attempts: {attempts})")

    # Save manifest
    manifest = pd.DataFrame(manifest_rows)
    manifest_path = out_root / "negative" / "negative_patches_manifest.csv"
    manifest.to_csv(manifest_path, index=False)

    print(f"\n✅ Negative Dataset Complete!")
    print(f"  Total patches: {len(manifest)}")
    print(f"  Static shape: ({len(static_vars)}, {tile_size}, {tile_size})")
    print(f"  Dynamic shape: ({window_days}, {len(dynamic_vars)}, {tile_size}, {tile_size})")
    if aligned_landcover:
        print(f"  Landcover shape: ({num_landcover_classes}, {tile_size}, {tile_size}) [one-hot encoded]")
    print(f"  Manifest: {manifest_path}")
    print(f"  Output directory: {out_root / 'negative'}")

    return manifest

In [3]:
config = {
    'ca_boundary': 'Dataset/Stataic Data/CA_State.gpkg',
    'fire_dataset': 'Dataset/Stataic Data/past_fire_2014_2024.gpkg',
    'fire_date_field': 'ALARM_DATE',
    
    'static_rasters': {
        'elevation': 'Dataset/Stataic Data/rasters_COP90/output_hh.tif',
        'slope': 'Dataset/Stataic Data/viz/Slope.tif',
        'population': 'Dataset/Stataic Data/Population/mosaic_masked.tif',
        'water_proximity': 'Dataset/Stataic Data/Waterway/ca_water_distance.tif',
        'road_proximity': 'Dataset/Stataic Data/roadways/ca_road_distance_snapped.tif',
    },
    'landcover_raster': 'Dataset/Stataic Data/LandCover/land_cover_cal_reclass.tif', 
    
    'dynamic_folders': {
        'relative_humidity': 'Dataset/Dynamic Data/relative_humidity',
        'total_precipitation': 'Dataset/Dynamic Data/Precipitation',
    },
    'reference_grid_path': 'Dataset/Stataic Data/rasters_COP90/elevation_fixed_3310_1000m_clipped.tif',
    'out_root': 'output_aligned_patches',
    'tile_size': 4,
    'window_days': 10,  
    'dst_crs': 'EPSG:3310',
    'dst_res': 1000.0,  
    'skip_existing_aligned': True,  
    'min_pos_frac': 0.10,  
    'require_all_valid': False,
    'num_landcover_classes': 10,
    'write_individual_landcover_tifs': True,
}

print("="*70)
print("ALIGNED FIRE DATASET BUILDER WITH LANDCOVER")
print("="*70)

print("\n[Step 0: Discovering Dynamic Rasters]")
dynamic_rasters = discover_dynamic_rasters(
    dynamic_folders=config['dynamic_folders'],
    pattern="*.tif"
)
print(f"\n✓ Total dynamic files found: {sum(len(files) for files in dynamic_rasters.values())}")

print("\n[Step 1: Creating Reference Grid]")
if os.path.exists(config['reference_grid_path']):
    print(f"  ✓ Using existing reference grid: {config['reference_grid_path']}")
    ref_grid = config['reference_grid_path']
else:
    ref_grid = create_reference_grid(
        ca_boundary=config['ca_boundary'],
        out_path=config['reference_grid_path'],
        dst_crs=config['dst_crs'],
        dst_res=config['dst_res']
    )

print("\n[Step 2: Aligning Rasters]")
print("⚠️  This may take a while with ~3,650 dynamic files (10 years × 365 days)")
print("    Set skip_existing_aligned=True to skip already processed files")
aligned_static, aligned_dynamic, aligned_landcover = align_all_rasters(
    static_rasters=config['static_rasters'],
    dynamic_rasters=dynamic_rasters,
    landcover_raster=config.get('landcover_raster'),  # NEW
    reference_grid=ref_grid,
    ca_boundary=config['ca_boundary'],
    out_root=config['out_root'],
    skip_existing=config['skip_existing_aligned']
)

print(f"\n✓ Alignment complete")
print(f"  Static: {len(aligned_static)} variables")
print(f"  Dynamic: {sum(len(files) for files in aligned_dynamic.values())} files across {len(aligned_dynamic)} variables")
if aligned_landcover:
    print(f"  Landcover: ✓ Aligned")

# print("\n[Step 3: Extracting Aligned Fire Patches]")
# print("  This will create .npy files for patches that overlap with fires")
# manifest = build_aligned_fire_dataset(
#     aligned_static=aligned_static,
#     aligned_dynamic=aligned_dynamic,
#     aligned_landcover=aligned_landcover,  # NEW
#     fire_dataset=config['fire_dataset'],
#     fire_date_field=config['fire_date_field'],
#     ca_boundary=config['ca_boundary'],
#     out_root=config['out_root'],
#     tile_size=config['tile_size'],
#     window_days=config['window_days'],
#     min_pos_frac=config['min_pos_frac'],
#     require_all_valid=config['require_all_valid'],
#     num_landcover_classes=config.get('num_landcover_classes', 10),  # NEW
#     write_individual_landcover_tifs=config.get('write_individual_landcover_tifs', True),  # NEW
# )

print("\n" + "="*70)


# =========================================================================
# STEP 4: Generate Negative Dataset (No Fire Samples)
# =========================================================================
print("\n" + "="*70)
print("GENERATING NEGATIVE DATASET")
print("="*70)

print("\n[Step 4: Extracting Negative Fire Patches]")
print("  This will create .npy files for patches with NO fire in 10-day window")
print("  Sampling 3000 patches per year (2015-2024)")

negative_manifest = build_negative_fire_dataset(
    aligned_static=aligned_static,
    aligned_dynamic=aligned_dynamic,
    aligned_landcover=aligned_landcover,
    fire_dataset=config['fire_dataset'],
    fire_date_field=config['fire_date_field'],
    ca_boundary=config['ca_boundary'],
    out_root=config['out_root'],
    tile_size=config['tile_size'],
    window_days=config['window_days'],
    samples_per_year=1000,
    min_valid_frac=config.get('min_valid_frac', 0.50),
    require_all_valid=config['require_all_valid'],
    num_landcover_classes=config.get('num_landcover_classes', 10),
    start_year=2015,
    end_year=2024,
)

print("\n" + "="*70)
print("📊 NEGATIVE DATASET SUMMARY")
print("="*70)
print(f"Total negative patches extracted: {len(negative_manifest)}")
print(f"\nStatic patch shape: ({len(aligned_static)}, {config['tile_size']}, {config['tile_size']})")
print(f"Dynamic patch shape: ({config['window_days']}, {len(aligned_dynamic)}, {config['tile_size']}, {config['tile_size']})")
if aligned_landcover:
    print(f"Landcover patch shape: ({config['num_landcover_classes']}, {config['tile_size']}, {config['tile_size']}) [one-hot encoded]")

print(f"\nOutput directory: {config['out_root']}/negative")
print(f"Manifest file: {config['out_root']}/negative/negative_patches_manifest.csv")

if len(negative_manifest) > 0:
    print(f"\n📝 First few patches:")
    print(negative_manifest.head(10))
    
    print(f"\n📅 Samples per year:")
    print(negative_manifest.groupby('year').size())

print("\n" + "="*70)
print("📊 SUMMARY")
print("="*70)
print(f"Total fire patches extracted: {len(manifest)}")
print(f"\nStatic patch shape: ({len(aligned_static)}, {config['tile_size']}, {config['tile_size']})")
print(f"Dynamic patch shape: ({config['window_days']}, {len(aligned_dynamic)}, {config['tile_size']}, {config['tile_size']})")
if aligned_landcover:
    print(f"Landcover patch shape: ({config['num_landcover_classes']}, {config['tile_size']}, {config['tile_size']}) [one-hot encoded]")

print(f"\nOutput directory: {config['out_root']}")
print(f"Manifest file: {config['out_root']}/aligned_patches_manifest.csv")

if len(manifest) > 0:
    print(f"\n📝 First few patches:")
    print(manifest.head(10))
    
    print(f"\n📅 Date range:")
    print(f"  Earliest: {manifest['fire_date'].min()}")
    print(f"  Latest: {manifest['fire_date'].max()}")

print("\n" + "="*70)

ALIGNED FIRE DATASET BUILDER WITH LANDCOVER

[Step 0: Discovering Dynamic Rasters]
  relative_humidity: found 3656 files
  total_precipitation: found 3653 files

✓ Total dynamic files found: 7309

[Step 1: Creating Reference Grid]
  ✓ Using existing reference grid: Dataset/Stataic Data/rasters_COP90/elevation_fixed_3310_1000m_clipped.tif

[Step 2: Aligning Rasters]
⚠️  This may take a while with ~3,650 dynamic files (10 years × 365 days)
    Set skip_existing_aligned=True to skip already processed files

[Aligning Static Rasters]
  ✓ Using existing: elevation_aligned.tif
  ✓ Using existing: slope_aligned.tif
  ✓ Using existing: population_aligned.tif
  ✓ Using existing: water_proximity_aligned.tif
  ✓ Using existing: road_proximity_aligned.tif

[Aligning Landcover Raster]
  ✓ Using existing: landcover_aligned.tif

[Aligning Dynamic Rasters]

  Processing relative_humidity: 3656 files


  Aligning relative_humidity: 100%|██████| 3656/3656 [00:00<00:00, 84074.65it/s]



  Processing total_precipitation: 3653 files


  Aligning total_precipitation: 100%|████| 3653/3653 [00:00<00:00, 97738.58it/s]


✓ Alignment complete
  Static: 5 variables
  Dynamic: 7309 files across 2 variables
  Landcover: ✓ Aligned


GENERATING NEGATIVE DATASET

[Step 4: Extracting Negative Fire Patches]
  This will create .npy files for patches with NO fire in 10-day window
  Sampling 3000 patches per year (2015-2024)

[Reference Grid] 915x1056 px


...

  Sampling year 2023: 100%|█████████████████| 1000/1000 [01:21<00:00, 12.26it/s]


  ✓ Year 2023: Generated 1000 negative samples (attempts: 1001)

[Processing Year 2024]
  Valid dates with 10-day window: 366


  Sampling year 2024: 100%|█████████████████| 1000/1000 [01:34<00:00, 10.54it/s]


  ✓ Year 2024: Generated 1000 negative samples (attempts: 1001)

✅ Negative Dataset Complete!
  Total patches: 10000
  Static shape: (5, 4, 4)
  Dynamic shape: (10, 2, 4, 4)
  Landcover shape: (10, 4, 4) [one-hot encoded]
  Manifest: output_aligned_patches/negative/negative_patches_manifest.csv
  Output directory: output_aligned_patches/negative

📊 NEGATIVE DATASET SUMMARY
Total negative patches extracted: 10000

Static patch shape: (5, 4, 4)
Dynamic patch shape: (10, 2, 4, 4)
Landcover patch shape: (10, 4, 4) [one-hot encoded]

Output directory: output_aligned_patches/negative
Manifest file: output_aligned_patches/negative/negative_patches_manifest.csv

📝 First few patches:
   patch_id                                        static_path  \
0         1  output_aligned_patches/negative/20151123_412_3...   
1         2  output_aligned_patches/negative/20150801_172_1...   
2         3  output_aligned_patches/negative/20150125_232_5...   
3         4  output_aligned_patches/negative/2015041

NameError: name 'manifest' is not defined